In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [26]:
df = pd.read_csv("../UH_2023/DATOS_METEO.txt", sep = "|")

In [27]:
df.validTimeUtc = pd.to_datetime(df.validTimeUtc)

Només necessitem dades horàries, que en tinguem suficients i pel 22.

In [28]:
df[df["windDirection"].isna()==True].validTimeUtc.dt.year.unique()

array([2015, 2016, 2017], dtype=int64)

In [29]:
columns_to_drop = ["precip2Day", "precip3Day", "precip7Day", "precipMtd", "precipYtd", "pressureMeanSeaLevel", "snow2Day", "snow3Day", "snow7Day", "snowMtd", "snowSeason", "snowYtd", "windDirection", "windGust", "precip24Hour", "precip6Hour", "snow24Hour", "snow6Hour", "temperatureChange24Hour", "temperatureMax24Hour", "temperatureMin24Hour", "visibility", "uvIndex"]

In [30]:
df2 = df.drop(columns_to_drop, axis=1)

In [31]:
cols_to_modify = list(df2.isna().sum()[df2.isna().sum() != 0].index)

Hi ha 20 files que falten.

In [32]:
df2[cols_to_modify] = (df2[cols_to_modify].ffill()+df2[cols_to_modify].bfill())/2

El 13/9/2015 hi ha 7 files que falten a totes les estacions.

In [33]:
for st in list(df2.ID_ESTACION.unique()):
    row_primer = df2[(df2.validTimeUtc.dt.day == 13) & ( df2.validTimeUtc.dt.month == 9) & (df2.validTimeUtc.dt.year == 2015) & (df2.ID_ESTACION == st)].iloc[-1, :]
    row_ultima = df2[(df2.validTimeUtc.dt.day == 14) & ( df2.validTimeUtc.dt.month == 9) & (df2.validTimeUtc.dt.year == 2015) & (df2.ID_ESTACION == st)].iloc[0, :]
    row_primer_del = row_primer.drop(['ID_ESTACION'])
    row_ultima_del = row_ultima.drop(['ID_ESTACION'])
    diff = row_ultima_del - row_primer_del
    diff /= 8
    for i in range(1, 8):
        df_aux = pd.DataFrame([row_primer_del + diff*i], columns=df2.columns[:-1])
        df_aux["ID_ESTACION"] = st
        df2 = pd.concat([df2, df_aux]).reset_index(drop = True)

### GroupBy amb day

In [34]:
df2_mean = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).mean()
df2_max = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).max()
df2_min = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).min()

In [35]:
cols_mean = [i + "DayAvg" for i in df2_mean.columns]
df2_mean.columns = cols_mean
cols_max = [i + "DayMax" for i in df2_max.columns]
df2_max.columns = cols_max
cols_min = [i + "DayMin" for i in df2_min.columns]
df2_min.columns = cols_min

### GroupyBy amb daytime

In [36]:
df2_daytime = df2[(7 < df2.validTimeUtc.dt.hour) & (df2.validTimeUtc.dt.hour < 19)]

df2_mean_daytime = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).mean()
df2_max_daytime = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).max()
df2_min_daytime = df2.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).min()

In [37]:
# Periodes normals
dic_periodes = {"Day": (0, 24), "Daytime": (7, 19), "Morning": (7, 13), "Afternoon": (13, 19)}
llista_df = {}
for k,(i,j) in dic_periodes.items():
    df_aux = df2[(i < df2.validTimeUtc.dt.hour) & (df2.validTimeUtc.dt.hour < j)]
    df2_mean = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).mean()
    df2_max = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).max()
    df2_min = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).min()
    
    cols_mean = [c + k + "Avg" for c in df2_mean.columns]
    df2_mean.columns = cols_mean
    cols_max = [c + k + "Max" for c in df2_max.columns]
    df2_max.columns = cols_max
    cols_min = [c + k + "Min" for c in df2_min.columns]
    df2_min.columns = cols_min
    llista_df[k] = [df2_mean, df2_max, df2_min]

In [38]:
# Periodes raros
dic_periodes = {"Nighttime": (19, 7), "Evening": (19, 1), "Overnight": (1, 7)}
for k,(i,j) in dic_periodes.items():
    df_aux = df2[(i < df2.validTimeUtc.dt.hour) | (df2.validTimeUtc.dt.hour < j)]
    df_aux.loc[:, "validTimeUtc"] = df_aux.loc[:, "validTimeUtc"] - pd.Timedelta(hours=10)
    df2_mean = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).mean()
    df2_max = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).max()
    df2_min = df_aux.groupby([pd.Grouper(key="validTimeUtc", freq="1D"), pd.Grouper(key = 'ID_ESTACION')]).min()
    
    cols_mean = [c + k + "Avg" for c in df2_mean.columns]
    df2_mean.columns = cols_mean
    cols_max = [c + k + "Max" for c in df2_max.columns]
    df2_max.columns = cols_max
    cols_min = [c + k + "Min" for c in df2_min.columns]
    df2_min.columns = cols_min
    llista_df[k] = [df2_mean, df2_max, df2_min]

C:\Users\xavid\AppData\Local\Temp\ipykernel_7368\3559150541.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux.loc[:, "validTimeUtc"] = df_aux.loc[:, "validTimeUtc"] - pd.Timedelta(hours=10)
C:\Users\xavid\AppData\Local\Temp\ipykernel_7368\3559150541.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux.loc[:, "validTimeUtc"] = df_aux.loc[:, "validTimeUtc"] - pd.Timedelta(hours=10)


In [39]:
first_df = llista_df["Day"][0]
llista = [j for j in [i for i in llista_df.values()]]
df_tractats = [item for sublist in llista for item in sublist]
df_tractats = df_tractats[1:]

In [40]:
for ds in df_tractats:
    first_df = pd.merge(first_df, ds, left_index=True, right_index=True)

In [41]:
first_df = first_df.reset_index()

In [42]:
first_df.to_csv("../DATASETS_TRACTATS/df_meteo_tractat.csv", index = False)